## Heart Attack Prediction

In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark =  SparkSession.builder.appName('Heart_attack').getOrCreate()

In [3]:
heart = spark.read.csv(r'C:\Users\Murillo\Desktop\Python\Portfolio\Heart_Attack\heart.csv', header=True, inferSchema=True)
heart.show(5)

+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
|age|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
| 63|  1|  3|   145| 233|  1|      0|     150|   0|    2.3|  0|  0|    1|     1|
| 37|  1|  2|   130| 250|  0|      1|     187|   0|    3.5|  0|  0|    2|     1|
| 41|  0|  1|   130| 204|  0|      0|     172|   0|    1.4|  2|  0|    2|     1|
| 56|  1|  1|   120| 236|  0|      1|     178|   0|    0.8|  2|  0|    2|     1|
| 57|  0|  0|   120| 354|  0|      1|     163|   1|    0.6|  2|  0|    2|     1|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
only showing top 5 rows



In [4]:
heart.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trtbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalachh: integer (nullable = true)
 |-- exng: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slp: integer (nullable = true)
 |-- caa: integer (nullable = true)
 |-- thall: integer (nullable = true)
 |-- output: integer (nullable = true)



In [5]:
df = heart.toPandas()
df.describe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
heart.drop_duplicates().show(5)

+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
|age|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
| 69|  0|  3|   140| 239|  0|      1|     151|   0|    1.8|  2|  2|    2|     1|
| 53|  0|  0|   130| 264|  0|      0|     143|   0|    0.4|  1|  0|    2|     1|
| 54|  1|  2|   125| 273|  0|      0|     152|   0|    0.5|  0|  1|    2|     1|
| 51|  1|  0|   140| 298|  0|      1|     122|   1|    4.2|  1|  3|    3|     0|
| 58|  0|  1|   136| 319|  1|      0|     152|   0|    0.0|  2|  2|    2|     0|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
only showing top 5 rows



In [7]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml import Pipeline

vec = VectorAssembler(inputCols=['sex', 'cp', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall'], outputCol='features_vec')
scaled = StandardScaler(inputCol='features_vec', outputCol='features_scaled')
pipeline = Pipeline(stages=[vec, scaled])
model = pipeline.fit(heart)
df = model.transform(heart)

In [8]:
heart_final = df.select('features_scaled', 'output')
heart_final.show(5)

+--------------------+------+
|     features_scaled|output|
+--------------------+------+
|[2.14587290662820...|     1|
|[2.14587290662820...|     1|
|[0.0,0.9689429657...|     1|
|[2.14587290662820...|     1|
|[0.0,0.0,7.116300...|     1|
+--------------------+------+
only showing top 5 rows



In [16]:
from pyspark.ml.classification import LogisticRegression

train_data, test_data = heart_final.randomSplit([0.8,0.2])
logreg = LogisticRegression(featuresCol='features_scaled', labelCol='output')
heart_train = logreg.fit(train_data)

In [17]:
test_data.groupBy('output').count().show()

+------+-----+
|output|count|
+------+-----+
|     1|   37|
|     0|   32|
+------+-----+



In [18]:
pred_result = heart_train.evaluate(test_data)

In [19]:
pred_result.predictions.show()

+--------------------+------+--------------------+--------------------+----------+
|     features_scaled|output|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(8,[1,2,4,7],[2.9...|     1|[-1.7102606193538...|[0.15312991529363...|       1.0|
|(8,[1,2,5,7],[0.9...|     1|[-2.5946671714110...|[0.06948242172160...|       1.0|
|(8,[1,2,5,7],[1.9...|     1|[-2.7986438114715...|[0.05739750578269...|       1.0|
|(8,[1,2,5,7],[1.9...|     1|[-3.2713041276511...|[0.03656885402692...|       1.0|
|(8,[1,2,5,7],[1.9...|     1|[-4.2522470505775...|[0.01403250379354...|       1.0|
|(8,[1,2,5,7],[1.9...|     1|[-4.2915336246074...|[0.01349920116354...|       1.0|
|(8,[2,3,5,7],[6.7...|     1|[-1.1067859967388...|[0.24847056354358...|       1.0|
|(8,[2,4,5,7],[5.4...|     1|[-0.3423005430304...|[0.41525075611924...|       1.0|
|(8,[2,4,5,7],[6.8...|     1|[-1.1340095231181...|[0.24342191685597...|       1.0|
|(8,

In [20]:
# Accuracy
pred_result.accuracy

0.8840579710144928

In [21]:
# True Positive Rate
pred_result.weightedTruePositiveRate

0.8840579710144928

In [22]:
# False Positive Rate
pred_result.weightedFalsePositiveRate

0.11716607912260088

In [23]:
# AUC ROC
pred_result.areaUnderROC

0.9273648648648647